In [185]:
class Bnode:
    def __init__(self,l1):
        self.keys=l1
        self.parent=None
        self.children=[]
        self.checkchild=False

In [186]:
def split_child(x,i,t):
    y=x.children[i]
    z=Bnode([])
    m=Bnode([])
    z.keys=(y.keys[0:t-1])
    m.keys=(y.keys[t:(2*t)-1])
    z.children[0:t]=y.children[t:(2*t)-1]
    r=x.children[len(x.keys)]
    x.keys.append([])
    x.children.append([])
    for j in range(len(x.keys)-1,i,-1):
        x.keys[j]=x.keys[j-1]
    x.keys[i]=y.keys[t-1]
    x.children[len(x.keys)]=r
    for j in range(len(x.keys)-1,i+1,-1):
        x.children[j]=x.children[j-1]
    x.children[i]=z
    x.children[i+1]=m
    for j in range(len(x.children)):
        x.children[j].parent=x
    return x

In [234]:
def merge(x,i,t):
    y=x.children[0]
    y.keys.append(x.keys[i])
    y.keys=y.keys+x.children[i+1].keys
    x.keys.pop(i)
    x.children.pop(i)
    x.children.pop(i)
    x.children.insert(i,y)
    if x.keys==[]:
        return x.children[0]
    return x

In [188]:
def leafinsert(x,k):
    if x.keys==[]:
        x.keys.append(k)
        return x
    x.keys.append([])
    j=len(x.keys)-1
    while j>0 and x.keys[j-1]>k:
        x.keys[j]=x.keys[j-1]
        j=j-1
    x.keys[j]=k
    return x

In [189]:
def nonleafinsert(x,k,t):
    if x.parent is None and len(x.keys)==(2*t)-1:
        z=Bnode([])
        z.checkchild=True
        z.children.append(x)
        z=split_child(z,0,t)
        for i in z.children:
            i.parent=z
        x=z
    if x.parent is None and x.checkchild==False:
        x=leafinsert(x,k)
        return x
    i=0
    while i<len(x.keys)-1 and x.keys[i]<k:
        i+=1
    if (x.checkchild==True) and len(x.children[i].keys)==(2*t)-1:
        x=split_child(x,i,t)
    if k<x.keys[i]:
        if (x.children[i].checkchild==False):
            new=leafinsert(x.children[i],k)
            return x
        else:
            nonleafinsert(x.children[i],k,t)
    else:
        if (x.children[i+1].checkchild==False):
            x=leafinsert(x.children[i+1],k)
            return x
        else:
            nonleafinsert(x.children[i+1],k,t)

In [235]:
l1=[]
l1=Bnode(l1)
# s=split_child(l1,1,3)
l1=nonleafinsert(l1,3,3)
l1=nonleafinsert(l1,6,3)
l1=nonleafinsert(l1,4,3)
l1=nonleafinsert(l1,5,3)
l1=nonleafinsert(l1,7,3)
l1=nonleafinsert(l1,2,3)
x=merge(l1,0,3)
print(x.keys)

[2, 3, 4, 5, 6, 7]


In [242]:
l1=[5]
l1=Bnode(l1)
l1.children.append(Bnode([3,4]))
l1.children.append(Bnode([6,7]))
x=merge(l1,0,3)
print(x.keys)

[3, 4, 5, 6, 7]


In [3]:
class Node:
    def __init__(self, data, par=None):
        # print ("Node __init__: " + str(data))
        self.data = list([data])
        self.parent = par
        self.child = list()

    def __str__(self):
        if self.parent:
            return str(self.parent.data) + ' : ' + str(self.data)
        return 'Root : ' + str(self.data)

    def __lt__(self, node):
        return self.data[0] < node.data[0]

    def _isLeaf(self):
        return len(self.child) == 0

    # merge new_node sub-tree into self node
    def _add(self, new_node):
        # print ("Node _add: " + str(new_node.data) + ' to ' + str(self.data))
        for child in new_node.child:
            child.parent = self
        self.data.extend(new_node.data)
        self.data.sort()
        self.child.extend(new_node.child)
        if len(self.child) > 1:
            self.child.sort()
        if len(self.data) > 2:
            self._split()

    # find correct node to insert new node into tree
    def _insert(self, new_node):
        # print ('Node _insert: ' + str(new_node.data) + ' into ' + str(self.data))

        # leaf node - add data to leaf and rebalance tree
        if self._isLeaf():
            self._add(new_node)

        # not leaf - find correct child to descend, and do recursive insert
        elif new_node.data[0] > self.data[-1]:
            self.child[-1]._insert(new_node)
        else:
            for i in range(0, len(self.data)):
                if new_node.data[0] < self.data[i]:
                    self.child[i]._insert(new_node)
                    break

    # 3 items in node, split into new sub-tree and add to parent
    def _split(self):
        # print("Node _split: " + str(self.data))
        left_child = Node(self.data[0], self)
        right_child = Node(self.data[2], self)
        if self.child:
            self.child[0].parent = left_child
            self.child[1].parent = left_child
            self.child[2].parent = right_child
            self.child[3].parent = right_child
            left_child.child = [self.child[0], self.child[1]]
            right_child.child = [self.child[2], self.child[3]]

        self.child = [left_child]
        self.child.append(right_child)
        self.data = [self.data[1]]

        # now have new sub-tree, self. need to add self to its parent node
        if self.parent:
            if self in self.parent.child:
                self.parent.child.remove(self)
            self.parent._add(self)
        else:
            left_child.parent = self
            right_child.parent = self

    # find an item in the tree; return item, or False if not found
    def _find(self, item):
        # print ("Find " + str(item))
        if item in self.data:
            return item
        elif self._isLeaf():
            return False
        elif item > self.data[-1]:
            return self.child[-1]._find(item)
        else:
            for i in range(len(self.data)):
                if item < self.data[i]:
                    return self.child[i]._find(item)

    def _remove(self, item):
        pass

    # print preorder traversal
    def _preorder(self):
        print(self)
        for child in self.child:
            child._preorder()


class Tree:
    def __init__(self):
        print("Tree __init__")
        self.root = None

    def insert(self, item):
        print("Tree insert: " + str(item))
        if self.root is None:
            self.root = Node(item)
        else:
            self.root._insert(Node(item))
            while self.root.parent:
                self.root = self.root.parent
        return True

    def find(self, item):
        return self.root._find(item)

    def remove(self, item):
        self.root.remove(item)

    def printTop2Tiers(self):
        print('----Top 2 Tiers----')
        print(str(self.root.data))
        for child in self.root.child:
            print(str(child.data), end=' ')
        print(' ')

    def preorder(self):
        print('----Preorder----')
        self.root._preorder()


tree = Tree()

lst = [13, 7, 24, 15, 4, 29, 20, 16, 19, 1, 5, 22, 17]
for item in lst:
    tree.insert(item)
tree.printTop2Tiers()

# for i in range (25):
# tree.insert(i)
# tree.printTop2Tiers()
# tree.preorder()
# print (tree.find(16))


Tree __init__
Tree insert: 13
Tree insert: 7
Tree insert: 24
Tree insert: 15
Tree insert: 4
Tree insert: 29
Tree insert: 20
Tree insert: 16
Tree insert: 19
Tree insert: 1
Tree insert: 5
Tree insert: 22
Tree insert: 17
----Top 2 Tiers----
[16]
[4, 13] [20, 24]  
